# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,whitehorse,60.7161,-135.0538,-22.57,77,75,1.03,CA,1738782596
1,1,lazaro cardenas,17.9583,-102.2000,29.21,51,0,3.58,MX,1738782717
2,2,bethel,41.3712,-73.4140,0.68,45,58,2.24,US,1738782719
3,3,waitangi,-43.9535,-176.5597,13.79,85,77,2.68,NZ,1738782720
4,4,adamstown,-25.0660,-130.1015,25.10,79,6,6.19,PN,1738782721


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color="City",
    alpha = 0.75)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values

# Ideal citities have max temperature lower than 27 degrees but higher than 21, Wind speed less than 4.5 m/s and zero cloudiness
ideal_city_data_df = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & 
                                     (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]


# Drop any rows with null values
ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,cabo san lucas,22.8909,-109.9124,25.17,38,0,3.60,MX,1738782602
81,81,crane,31.3974,-102.3501,26.82,29,0,4.35,US,1738782809
139,139,cedral,23.8000,-100.7333,22.23,37,0,1.16,MX,1738782877
172,172,hawaiian paradise park,19.5933,-154.9731,23.83,81,0,2.06,US,1738782916
265,265,geraldton,-28.7667,114.6000,23.23,88,0,3.09,AU,1738782820
290,290,juan aldama,24.2908,-103.3944,23.48,14,0,2.80,MX,1738783057
403,403,srivardhan,18.0333,73.0167,22.85,60,0,2.56,IN,1738783191
450,450,gumel,12.6279,9.3912,23.92,8,0,4.19,NG,1738783244
467,467,port blair,11.6667,92.7500,25.07,79,0,3.40,IN,1738783263
524,524,gautier,30.3858,-88.6117,24.48,70,0,3.09,US,1738783327


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
9,cabo san lucas,MX,22.8909,-109.9124,38,
81,crane,US,31.3974,-102.3501,29,
139,cedral,MX,23.8000,-100.7333,37,
172,hawaiian paradise park,US,19.5933,-154.9731,81,
265,geraldton,AU,-28.7667,114.6000,88,
290,juan aldama,MX,24.2908,-103.3944,14,
403,srivardhan,IN,18.0333,73.0167,60,
450,gumel,NG,12.6279,9.3912,8,
467,port blair,IN,11.6667,92.7500,79,
524,gautier,US,30.3858,-88.6117,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories' : 'accommodation.hotel',
    'apiKey' : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row ['Lat']
    lng = row ['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()


    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
crane - nearest hotel: No hotel found
cedral - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
geraldton - nearest hotel: Ocean Centre Hotel
juan aldama - nearest hotel: No hotel found
srivardhan - nearest hotel: No hotel found
gumel - nearest hotel: No hotel found
port blair - nearest hotel: Andaman Ocean Breeze
gautier - nearest hotel: Grand Magnolia


,City,Country,Lat,Lng,Humidity,Hotel Name
9,cabo san lucas,MX,22.8909,-109.9124,38,Comfort Rooms
81,crane,US,31.3974,-102.3501,29,No hotel found
139,cedral,MX,23.8000,-100.7333,37,No hotel found
172,hawaiian paradise park,US,19.5933,-154.9731,81,No hotel found
265,geraldton,AU,-28.7667,114.6000,88,Ocean Centre Hotel
290,juan aldama,MX,24.2908,-103.3944,14,No hotel found
403,srivardhan,IN,18.0333,73.0167,60,No hotel found
450,gumel,NG,12.6279,9.3912,8,No hotel found
467,port blair,IN,11.6667,92.7500,79,Andaman Ocean Breeze
524,gautier,US,30.3858,-88.6117,70,Grand Magnolia


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color="City",
    alpha = 0.75,
    hover_cols= ["Hotel Name", "Country"])

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)